In [1]:
using JuMP
import HiGHS
import GLPK
using Clp

## Lista 1

### Questão 1

In [2]:
#model = Model(HiGHS.Optimizer)
model = Model(GLPK.Optimizer)

i = 2
j = 3
weights = [1, 1.5, 3.5, 2, 1, 2]

@variable(model, x[1:i, 1:j] >= 0)
@constraint(model, x[1,1] + x[1,2] + x[1,3] <= 400)
@constraint(model, x[2,1] + x[2,2] + x[2,3] <= 300)
@constraint(model, x[1,1] + x[2,1]   == 100)
@constraint(model, x[1,2] + x[2,2]  == 200)
@constraint(model, x[1,3] + x[2,3]  == 300)

#@objective(model, Min, sum(weights[n] * x[1:i, 1:j] for n in 1:6))
@objective(model, Min, sum(weights[n] * x[div(n-1,3)+1, mod(n-1,3)+1] for n in 1:length(weights)))

x[1,1] + 1.5 x[1,2] + 3.5 x[1,3] + 2 x[2,1] + x[2,2] + 2 x[2,3]

In [3]:
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))

for m in 1:i
    for n in 1:j
        println("x[$m, $n] = ", round(Int, value(x[m,n])))
    end
end

Valor Objetivo (Custo Mínimo): 1000.0
x[1, 1] = 100
x[1, 2] = 200
x[1, 3] = 0
x[2, 1] = 0
x[2, 2] = 0
x[2, 3] = 300


Minimum Cost: 1000 

Generalizando para todos os casos:

$$ \begin{align} \min \; & c^T x \\
s.t \; & Ax = b \\
& Cx \leq d \\
& x \geq 0 
\end{align}$$

In [4]:
function LPP_Transporte(i, j, weights, A, C, b, d)
    model = Model(GLPK.Optimizer)
    p = length(weights)
    @variable(model, x[1:i, 1:j] >= 0)
    x = reshape(x, p, 1)
    for m in 1:j
        @constraint(model, (A*x)[m] == b[m])
    end
    for n in 1:i
        @constraint(model, (C*x)[n] <= d[n])
    end

    @objective(model, Min, sum(weights[k] * x[k] for k in 1:p))
    println("Função Objetivo: ", objective_function(model))
    optimize!(model)
    for k in 1:p
        m,n = div(k-1, j) + 1, mod(k-1, j) + 1
        println("x[$m, $n] = " , value(x[k]))
    end
    println("Valor Objetivo (Custo Mínimo): ", objective_value(model))
end

LPP_Transporte (generic function with 1 method)

Exemplificando com o problema anterior...

In [5]:
A = [1 0 0 1 0 0;
0 1 0 0 1 0;
0 0 1 0 0 1]
b = [100;200;300]

C = [1 1 1 0 0 0;
 0 0 0 1 1 1]
d = [400; 300]

weights = [1 1.5 3.5 2 1 2]
i = 2
j = 3;

In [6]:
LPP_Transporte(i, j, weights, A, C, b, d)

Função Objetivo: x[1,1] + 1.5 x[2,1] + 3.5 x[1,2] + 2 x[2,2] + x[1,3] + 2 x[2,3]
x[1, 1] = 100.0
x[1, 2] = 200.0
x[1, 3] = 0.0
x[2, 1] = 0.0
x[2, 2] = 0.0
x[2, 3] = 300.0
Valor Objetivo (Custo Mínimo): 1000.0


## Lista 2

### Problema da Mochila

In [7]:
n = 5;
capacity = 10.0;
profit = [5.0, 3.0, 2.0, 7.0, 4.0];
weight = [2.0, 8.0, 4.0, 2.0, 5.0];

#model = Model(GLPK.Optimizer)
model = Model(HiGHS.Optimizer)

@variable(model, x[1:n], Bin)

@constraint(model, sum(weight[i] * x[i] for i in 1:n) <= capacity)

@objective(model, Max, sum(profit[i] * x[i] for i in 1:n))

5 x[1] + 3 x[2] + 2 x[3] + 7 x[4] + 4 x[5]

LPP:

In [8]:
print(model)

Max 5 x[1] + 3 x[2] + 2 x[3] + 7 x[4] + 4 x[5]
Subject to
 2 x[1] + 8 x[2] + 4 x[3] + 2 x[4] + 5 x[5] <= 10
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary


In [9]:
#set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e+00, 8e+00]
  Cost   [2e+00, 7e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+01, 1e+01]
Presolving model
1 rows, 5 cols, 5 nonzeros  0s
1 rows, 4 cols, 4 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   1 rows
   4 cols (4 binary, 0 integer, 0 implied int., 0 continuous)
   4 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   18              -inf                 inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      16
  Dual bound        16
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    16 (objec

In [10]:
items_chosen = [i for i in 1:n if value(x[i]) == 1]
items_chosen

16.0


3-element Vector{Int64}:
 1
 4
 5

In [11]:
for i in 1:n
    print("x[$i] = ", round(Int, value(x[i])))
    println(", c[$i] / w[$i] = ", profit[i] / weight[i])
end

x[1] = 1, c[1] / w[1] = 2.5
x[2] = 0, c[2] / w[2] = 0.375


Maximum Profit = 16 \
Choosen Objects = 1,4,5

## Prova A1

### Questão 2

In [15]:
model = Model(HiGHS.Optimizer)

i = 3
j = 3
weights = [50, 90, 80, 0, 30, 0, 0, 40, 50]

@variable(model, x[1:i, 1:j] >= 0)

@constraint(model, x[1,1] == 20)
@constraint(model, x[1,2] + x[2,2] + x[3,2] == 10)
@constraint(model, x[1,3] + x[2,3] + x[3,3]  == 30)
@constraint(model, x[1,1] + x[1,2] + x[1,3]  <= 50)
@constraint(model, x[2,2] + x[2,3]  <= 50)
@constraint(model, x[3,2] + x[3,3]  <= 20)

#@objective(model, Min, sum(weights[n] * x[1:i, 1:j] for n in 1:6))
@objective(model, Min, sum(weights[n] * x[div(n-1,3)+1, mod(n-1,3)+1] for n in 1:9))

50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 30 x[2,2] + 40 x[3,2] + 50 x[3,3]

In [17]:
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
print("Valor Objetivo (Custo Mínimo): ", objective_value(model))

Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 9e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+01, 5e+01]
Solving LP without presolve, or with basis, or unconstrained
Model   status      : Optimal
Objective value     :  1.3000000000e+03
HiGHS run time      :          0.00
Valor Objetivo (Custo Mínimo): 1300.0

In [18]:
for i in 1:3
    for j in 1:3
        println("x[$i, $j] = ", round(Int, value(x[i,j])))
    end
end

x[1, 1] = 20
x[1, 2] = 0
x[1, 3] = 0
x[2, 1] = 0
x[2, 2] = 10
x[2, 3] = 30
x[3, 1] = 0
x[3, 2] = 0
x[3, 3] = 0


Minimum Cost: 1300 \
Observations: Ilumination recieves only from eletricity, and space/water heating recieve from solar painels (its the cheapiest)